# Practice 1. Критерии Стьюдента

In [42]:
from scipy import stats
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import pandas as pd
from statsmodels.stats.descriptivestats import sign_test

## Question 4

Уровень кальция в крови здоровых молодых женщин равен в среднем 9.5 милиграммам на децилитр и имеет характерное стандартное отклонение 0.4 мг/дл. В сельской больнице Гватемалы для 160 здоровых беременных женщин при первом обращении для ведения беременности был измерен уровень кальция; среднее значение составило 9.57 мг/дл. Можно ли утверждать, что средний уровень кальция в этой популяции отличается от 9.5?

Посчитайте достигаемый уровень значимости. Поскольку известны только среднее и дисперсия, а не сама выборка, нельзя использовать стандартные функции критериев — нужно реализовать формулу достигаемого уровня значимости самостоятельно.

Округлите ответ до четырёх знаков после десятичной точки.

In [77]:
z = (9.57 - 9.5)*np.sqrt(160)/.4
print('p =', round(2*(1 - stats.norm.cdf(abs(z))), 4))

p = 0.0269


# Quiz 1. Параметрические критерии 

## Question 3 

В одном из выпусков программы "Разрушители легенд" проверялось, действительно ли заразительна зевота. В эксперименте участвовало 50 испытуемых, проходивших собеседование на программу. Каждый из них разговаривал с рекрутером; в конце 34 из 50 бесед рекрутер зевал. Затем испытуемых просили подождать решения рекрутера в соседней пустой комнате.

Во время ожидания 10 из 34 испытуемых экспериментальной группы и 4 из 16 испытуемых контрольной начали зевать. Таким образом, разница в доле зевающих людей в этих двух группах составила примерно 4.4%. Ведущие заключили, что миф о заразительности зевоты подтверждён.

Можно ли утверждать, что доли зевающих в контрольной и экспериментальной группах отличаются статистически значимо? Посчитайте достигаемый уровень значимости при альтернативе заразительности зевоты, округлите до четырёх знаков после десятичной точки.

### Z-критерий для разности долей (независимые выборки)

   | $X_1$ | $X_2$  
  ------------- | -------------|
  1  | a | b 
  0  | c | d 
  $\sum$ | $n_1$| $n_2$
  
$$ \hat{p}_1 = \frac{a}{n_1}$$

$$ \hat{p}_2 = \frac{b}{n_2}$$


$$\text{Доверительный интервал для }p_1 - p_2\colon \;\; \hat{p}_1 - \hat{p}_2 \pm z_{1-\frac{\alpha}{2}}\sqrt{\frac{\hat{p}_1(1 - \hat{p}_1)}{n_1} + \frac{\hat{p}_2(1 - \hat{p}_2)}{n_2}}$$

$$Z-статистика: Z({X_1, X_2}) =  \frac{\hat{p}_1 - \hat{p}_2}{\sqrt{P(1 - P)(\frac{1}{n_1} + \frac{1}{n_2})}}$$
$$P = \frac{\hat{p}_1{n_1} + \hat{p}_2{n_2}}{{n_1} + {n_2}} $$

In [6]:
n1 = 34
n2 = 16

a = 10
b = 4

1.5

In [7]:
def z_stat_bern(n1 = n1, n2 = n2, a = a, b = b):
    p1 = a/n1
    p2 = b/n2
    
    P = (a + b)/(n1 + n2)
    
    return (p1 - p2)/(np.sqrt(P*(1-P)*(1/n1 + 1/n2)))

In [8]:
def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - stats.norm.cdf(z_stat)

In [129]:
print('Нулевая гипотеза: средняя доля зевающих в обоих выборках статистически значимо не различается\n\n\
H0: p1 = p2\n\
H1: p1 > p2\n\n\
p =', round(proportions_diff_z_test(z_stat_bern(), alternative = 'greater'), 4))

Нулевая гипотеза: средняя доля зевающих в обоих выборках статистически значимо не различается

H0: p1 = p2
H1: p1 > p2

p = 0.3729


## Question 4 

Имеются данные измерений двухсот швейцарских тысячефранковых банкнот, бывших в обращении в первой половине XX века. Сто из банкнот были настоящими, и сто — поддельными. 

Отделите 50 случайных наблюдений в тестовую выборку с помощью функции sklearn.cross_validation.train_test_split (зафиксируйте random state = 1). На оставшихся 150 настройте два классификатора поддельности банкнот:
* логистическая регрессия по признакам $X_1,X_2,X_3$ 
* логистическая регрессия по признакам $X_4,X_5,X_6$ 

Каждым из классификаторов сделайте предсказания меток классов на тестовой выборке. Одинаковы ли доли ошибочных предсказаний двух классификаторов? Проверьте гипотезу, вычислите достигаемый уровень значимости. Введите номер первой значащей цифры (например, если вы получили $5.5\times10^{-8}$, нужно ввести 8).

In [22]:
banknotes = pd.read_csv('banknotes.txt', sep = '\t')
banknotes.head()

,X1,X2,X3,X4,X5,X6,real
0,214.8,131.0,131.1,9.0,9.7,141.0,1
1,214.6,129.7,129.7,8.1,9.5,141.7,1
2,214.8,129.7,129.7,8.7,9.6,142.2,1
3,214.8,129.7,129.6,7.5,10.4,142.0,1
4,215.0,129.6,129.7,10.4,7.7,141.8,1


In [25]:
X = banknotes.loc[:, banknotes.columns != 'real']
y = banknotes['real']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 50, random_state = 1)

print(X_train.shape, X_test.shape)

(150, 6) (50, 6)


In [54]:
L1 = LogisticRegression().fit(X_train.iloc[:, :3], y_train).predict(X_test.iloc[:, :3])
L2 = LogisticRegression().fit(X_train.iloc[:, 3:], y_train).predict(X_test.iloc[:, 3:])

print('p1 = {}\np2 = {}'.format(sum(L1)/len(L1), sum(L2)/len(L2)))

p1 = 0.46
p2 = 0.56


### Z-критерий для разности долей (связанные выборки)

  $X_1$ \ $X_2$ | 1| 0 | $\sum$
  ------------- | -------------|
  1  | e | f | e + f
  0  | g | h | g + h
  $\sum$ | e + g| f + h | n  
  
$$ \hat{p}_1 = \frac{e + f}{n}$$

$$ \hat{p}_2 = \frac{e + g}{n}$$

$$ \hat{p}_1 - \hat{p}_2 = \frac{f - g}{n}$$


$$\text{Доверительный интервал для }p_1 - p_2\colon \;\;  \frac{f - g}{n} \pm z_{1-\frac{\alpha}{2}}\sqrt{\frac{f + g}{n^2} - \frac{(f - g)^2}{n^3}}$$

$$Z-статистика: Z({X_1, X_2}) = \frac{f - g}{\sqrt{f + g - \frac{(f-g)^2}{n}}}$$

In [109]:
def proportions_diff_z_stat_rel(sample1, sample2):
    sample = list(zip(sample1, sample2))
    n = len(sample)
    
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    return float(f - g) / np.sqrt(f + g - float((f - g)**2) / n)

In [128]:
print('Нулевая гипотеза: средняя доля ошибочных предсказаний в обоих моделях статистически значимо не различается\n\n\
H0: p1 = p2\n\
H1: p1 != p2\n\n\
p =', proportions_diff_z_test(proportions_diff_z_stat_rel(y_test != L1, y_test != L2))) 
print('\nОтвет:', 3)

Нулевая гипотеза: средняя доля ошибочных предсказаний в обоих моделях статистически значимо не различается

H0: p1 = p2
H1: p1 != p2

p = 0.00329693845555

Ответ: 3


## Question 5 

В предыдущей задаче посчитайте 95% доверительный интервал для разности долей ошибок двух классификаторов. Чему равна его ближайшая к нулю граница? Округлите до четырёх знаков после десятичной точки.

In [113]:
def proportions_diff_confint_rel(sample1, sample2, alpha = 0.05):
    z = stats.norm.ppf(1 - alpha / 2.)
    sample = list(zip(sample1, sample2))
    n = len(sample)
        
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    left_boundary = float(f - g) / n  - z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    right_boundary = float(f - g) / n  + z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    return (left_boundary, right_boundary)

In [124]:
print('Доверительный интервал для связанных выборок', proportions_diff_confint_rel(y_test != L1, y_test != L2, alpha = 0.05))
print('Ответ:', 0.0599)

Доверительный интервал для связанных выборок (0.059945206279614305, 0.30005479372038568)
Ответ: 0.0599


## Question 6

Ежегодно более 200000 людей по всему миру сдают стандартизированный экзамен GMAT при поступлении на программы MBA. Средний результат составляет 525 баллов, стандартное отклонение — 100 баллов.

Сто студентов закончили специальные подготовительные курсы и сдали экзамен. Средний полученный ими балл — 541.4. Проверьте гипотезу о неэффективности программы против односторонней альтернативы о том, что программа работает. Отвергается ли на уровне значимости 0.05 нулевая гипотеза? Введите достигаемый уровень значимости, округлённый до 4 знаков после десятичной точки.

In [121]:
MU = 525
SD = 100

n = 100
mu = 541.4
alpha = .05

print('H0: mu = MU\nH1: mu > MU')

H0: mu = MU
H1: mu > MU


In [122]:
def z_criteria(mu = mu, MU = MU, SD = SD, n = n):
    return (mu - MU)*np.sqrt(n)/SD

In [123]:
print('p =', round(1 - stats.norm.cdf(z_criteria()), 4))

p = 0.0505


# Quiz 2. Непараметрические критерии

## Question 4

Давайте вернёмся к данным выживаемости пациентов с лейкоцитарной лимфомой из видео про критерий знаков:

49, 58, 75, 110, 112, 132, 151, 276, 281, 362*
 
Измерено остаточное время жизни с момента начала наблюдения (в неделях); звёздочка обозначает цензурирование сверху — исследование длилось 7 лет, и остаточное время жизни одного пациента, который дожил до конца наблюдения, неизвестно.

Поскольку цензурировано только одно наблюдение, для проверки гипотезы $H_0\colon med X = 200$ на этих данных можно использовать критерий знаковых рангов — можно считать, что время дожития последнего пациента в точности равно 362, на ранг этого наблюдения это никак не повлияет.

Критерием знаковых рангов проверьте эту гипотезу против двусторонней альтернативы, введите достигаемый уровень значимости, округлённый до четырёх знаков после десятичной точки.

In [49]:
m0 = 200
leyk = np.array([49, 58, 75, 110, 112, 132, 151, 276, 281, 362])

leyk_test = stats.wilcoxon(leyk - m0)

print("M: {}, p-value: {}".format(leyk_test[0], round(leyk_test[1], 4)))

M: 17.0, p-value: 0.2845


## Question 5

В ходе исследования влияния лесозаготовки на биоразнообразие лесов острова Борнео собраны данные о количестве видов деревьев в 12 лесах, где вырубка не ведётся:

$22, 22, 15, 13, 19, 19, 18, 20, 21, 13, 13, 15,22,22,15,13,19,19,18,20,21,13,13,15$

и в 9 лесах, где идёт вырубка:

$17, 18, 18, 15, 12, 4, 14, 15, 10.17,18,18,15,12,4,14,15,10$.

Проверьте гипотезу о равенстве среднего количества видов в двух типах лесов против односторонней альтернативы о снижении биоразнообразия в вырубаемых лесах. Используйте ранговый критерий. Чему равен достигаемый уровень значимости? Округлите до четырёх знаков после десятичной точки.

In [119]:
#независимые выборки
s1 = [22, 22, 15, 13, 19, 19, 18, 20, 21, 13, 13, 15,22,22,15,13,19,19,18,20,21,13,13,15]
s2 = [17, 18, 18, 15, 12, 4, 14, 15, 10.17,18,18,15,12,4,14,15,10]

forest_test = stats.mannwhitneyu(s2, s1, alternative = 'greater')

print('Statistics: {}, p-value: {}'.format(forest_test[0], round(forest_test[1], 4)))

Statistics: 98.0, p-value: 0.9977


## Question 6

28 января 1986 года космический шаттл "Челленджер" взорвался при взлёте. Семь астронавтов, находившихся на борту, погибли. В ходе расследования причин катастрофы основной версией была неполадка с резиновыми уплотнительными кольцами в соединении с ракетными ускорителями. Для 23 предшествовавших катастрофе полётов "Челленджера" известны температура воздуха и появление повреждений хотя бы у одного из уплотнительных колец.

С помощью бутстрепа постройте 95% доверительный интервал для разности средних температур воздуха при запусках, когда уплотнительные кольца повреждались, и запусках, когда повреждений не было. Чему равна его ближайшая к нулю граница? Округлите до четырёх знаков после запятой.

Чтобы получить в точности такой же доверительный интервал, как у нас:

* установите random seed = 0 перед первым вызовом функции get_bootstrap_samples, один раз
* сделайте по 1000 псевдовыборок из каждой выборки.

In [51]:
challenger = pd.read_csv('challenger.txt', sep = '\t')
challenger.head()

,Unnamed: 0,Temperature,Incident
0,Apr12.81,18.9,0
1,Nov12.81,21.1,1
2,Mar22.82,20.6,0
3,Nov11.82,20.0,0
4,Apr04.83,19.4,0


In [67]:
fail = challenger[(challenger['Incident'] == 1)]['Temperature'].values
alive = challenger[(challenger['Incident'] == 0)]['Temperature'].values

In [85]:
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

In [86]:
def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

In [93]:
np.random.seed(0)

alive_mean_scores = list(map(np.mean, get_bootstrap_samples(alive, 1000)))
fail_mean_scores = list(map(np.mean, get_bootstrap_samples(fail, 1000)))

delta_mean_scores = list(map(lambda x: x[1] - x[0], zip(fail_mean_scores, alive_mean_scores)))

In [115]:
print("95% confidence interval for the difference between means",  stat_intervals(delta_mean_scores, 0.05))
print('Answer is', round(stat_intervals(delta_mean_scores, 0.05)[0], 4))

95% confidence interval for the difference between means [ 1.42299107  7.93861607]
Answer is 1.423


## Question 7

На данных предыдущей задачи проверьте гипотезу об одинаковой средней температуре воздуха в дни, когда уплотнительный кольца повреждались, и дни, когда повреждений не было. Используйте перестановочный критерий и двустороннюю альтернативу. Чему равен достигаемый уровень значимости? Округлите до четырёх знаков после десятичной точки.

Чтобы получить такое же значение, как мы:

* установите random seed = 0;
* возьмите 10000 перестановок.

In [101]:
def permutation_t_stat_ind(sample1, sample2):
    return np.mean(sample1) - np.mean(sample2)

In [110]:
def get_random_combinations(n1, n2, max_combinations):
    index = list(range(n1 + n2))
    indices = set([tuple(index)])
    for i in range(max_combinations - 1):
        np.random.shuffle(index)
        indices.add(tuple(index))
    return [(index[:n1], index[n1:]) for index in indices]

In [103]:
def permutation_zero_dist_ind(sample1, sample2, max_combinations = None):
    joined_sample = np.hstack((sample1, sample2))
    n1 = len(sample1)
    n = len(joined_sample)
    
    if max_combinations:
        indices = get_random_combinations(n1, len(sample2), max_combinations)
    else:
        indices = [(list(index), filter(lambda i: i not in index, range(n))) \
                    for index in itertools.combinations(range(n), n1)]
    
    distr = [joined_sample[list(i[0])].mean() - joined_sample[list(i[1])].mean() \
             for i in indices]
    return distr

In [104]:
def permutation_test(sample, mean, max_permutations = None, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    t_stat = permutation_t_stat_ind(sample, mean)
    
    zero_distr = permutation_zero_dist_ind(sample, mean, max_permutations)
    
    if alternative == 'two-sided':
        return sum([1. if abs(x) >= abs(t_stat) else 0. for x in zero_distr]) / len(zero_distr)
    
    if alternative == 'less':
        return sum([1. if x <= t_stat else 0. for x in zero_distr]) / len(zero_distr)

    if alternative == 'greater':
        return sum([1. if x >= t_stat else 0. for x in zero_distr]) / len(zero_distr)

In [116]:
np.random.seed(0)

print('p-value =', permutation_test(fail, alive, max_permutations = 10000))

p-value = 0.0057
